## Loading dataset

***Load the CSV file***

In [38]:
import pandas as pd

# Load CSV
csv_path = "Dataset/metadata.csv"
df = pd.read_csv(csv_path)

# Inspect data
print(df.head())

  image_id                   domain split     image_path
0        1  B (Facade Segmentation)  test    testB/1.jpg
1        1  B (Facade Segmentation)  test    testB/1.jpg
2       10  B (Facade Segmentation)  test   testB/10.jpg
3       10  B (Facade Segmentation)  test   testB/10.jpg
4      100  B (Facade Segmentation)  test  testB/100.jpg


***Filter and split the dataset***

In [39]:
train_data = df[df['split'] == 'train']
test_data = df[df['split'] == 'test']

print(f"Train samples: {len(train_data)}, Test samples: {len(test_data)}")


Train samples: 800, Test samples: 212


***Organize paths***

In [40]:
trainA_paths = train_data[train_data['domain'] == 'A (Facade Photo)']['image_path'].tolist()
trainB_paths = train_data[train_data['domain'] == 'B (Facade Segmentation)']['image_path'].tolist()

testA_paths = test_data[test_data['domain'] == 'A (Facade Photo)']['image_path'].tolist()
testB_paths = test_data[test_data['domain'] == 'B (Facade Segmentation)']['image_path'].tolist()

***Load and preprocess images***

In [41]:
from PIL import Image
import numpy as np
import os

def load_images(paths, base_folder, size=(128, 128)):
    images = []
    for path in paths:
        if os.path.isfile(os.path.join(base_folder, path)):  # Ensure the file exists
            img = Image.open(os.path.join(base_folder, path)).resize(size)
            images.append(np.array(img) / 255.0)  # Normalize
    return np.array(images)

# Filter paths to include only valid ones for trainA and trainB
trainA_paths = train_data[train_data['image_path'].str.contains("A.jpg")]['image_path'].tolist()
trainB_paths = train_data[train_data['image_path'].str.contains("B.jpg")]['image_path'].tolist()

testA_paths = test_data[test_data['image_path'].str.contains("A.jpg")]['image_path'].tolist()
testB_paths = test_data[test_data['image_path'].str.contains("B.jpg")]['image_path'].tolist()

# Load train and test images
trainA_images = load_images(trainA_paths, base_folder="Dataset")
trainB_images = load_images(trainB_paths, base_folder="Dataset")

testA_images = load_images(testA_paths, base_folder="Dataset")
testB_images = load_images(testB_paths, base_folder="Dataset")


## Training the Model

In [42]:
import tensorflow as tf
from tensorflow_examples.models.pix2pix import pix2pix

# Define the generator (U-Net architecture for RGB output)
generator = pix2pix.unet_generator(3, norm_type='batchnorm')

# Define the discriminator (PatchGAN)
discriminator = pix2pix.discriminator(norm_type='batchnorm', target=False)

# Optimizers
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

# Loss function
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)


In [43]:
print(trainB_images.shape)
print(trainA_images.shape)


(294, 128, 128, 3)
(400, 128, 128, 3)


In [44]:
min_size = min(len(trainB_images), len(trainA_images))
trainB_images = trainB_images[:min_size]
trainA_images = trainA_images[:min_size]


In [45]:
min_test_size = min(len(testB_images), len(testA_images))
testB_images = testB_images[:min_test_size]
testA_images = testA_images[:min_test_size]


In [46]:
BUFFER_SIZE = 400
BATCH_SIZE = 16

# Create training and test datasets
train_dataset = tf.data.Dataset.from_tensor_slices((trainB_images, trainA_images))
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices((testB_images, testA_images))
test_dataset = test_dataset.batch(BATCH_SIZE)

# Training loop
EPOCHS = 10

for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")
    for input_image, target in train_dataset:
        gen_loss, disc_loss = train_step(input_image, target)
    print(f"Generator Loss: {gen_loss.numpy()}, Discriminator Loss: {disc_loss.numpy()}")


Epoch 1/10


ValueError: in user code:

    File "C:\Users\pc\AppData\Local\Temp\ipykernel_14592\3033065970.py", line 19, in train_step  *
        gen_output = generator(input_image, training=True)
    File "c:\Users\pc\pyver\py3123\Lib\site-packages\keras\src\utils\traceback_utils.py", line 122, in error_handler  **
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling Concatenate.call().
    
    [1mDimension 1 in both shapes must be equal, but are 2 and 1. Shapes are [16,2,2] and [16,1,1]. for '{{node functional_148_1/concatenate_6_1/concat}} = ConcatV2[N=2, T=DT_FLOAT, Tidx=DT_INT32](functional_148_1/sequential_116_1/re_lu_42_1/Relu, functional_148_1/sequential_114_1/leaky_re_lu_78_1/LeakyRelu, functional_148_1/concatenate_6_1/concat/axis)' with input shapes: [16,2,2,512], [16,1,1,512], [] and with computed input tensors: input[2] = <-1>.[0m
    
    Arguments received by Concatenate.call():
      • inputs=['tf.Tensor(shape=(16, 2, 2, 512), 

## Evaluation

In [47]:
import matplotlib.pyplot as plt

# Generate and visualize results for 3 test images
for i in range(3):
    input_image = testB_images[i][None, ...]  # Add batch dimension
    prediction = generator(input_image, training=False)

    # Plot the input, prediction, and ground truth
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.title("Input (Sketch)")
    plt.imshow(tf.keras.utils.array_to_img(input_image[0]))
    plt.subplot(1, 3, 2)
    plt.title("Generated Image")
    plt.imshow(tf.keras.utils.array_to_img(prediction[0]))
    plt.subplot(1, 3, 3)
    plt.title("Ground Truth")
    plt.imshow(tf.keras.utils.array_to_img(testA_images[i]))
    plt.show()


IndexError: index 0 is out of bounds for axis 0 with size 0